In [1]:
import random

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [2]:
from queue import PriorityQueue

def compute_cost(I, Si):
    '''
        I represents set of current elements included in the solution.  Initially I = {}
        Si represents i-th subset taken into consideration in order to be added to the solution
        |Si - I| is the number of new elements added by Si
        the first cost is inversely proportional to the sum between |Si - I| and the number of elements already into I,
        while the second one represent the number of repetition added by the insertion of Si
        return -1 if Si do not add any element (|Si - I| = 0), the cost otherwise
    '''
    # compute how many elements of I are already inside Si 
    nI = 0
    for i in I:
        if i in Si:
            nI += 1 
    # compute how many new elements will be added 
    n_new_el = abs(len(Si) - nI)

    if n_new_el == 0:
        return -1
    else:
        return (1/(n_new_el + len(I)) , nI)


def tree_search(blocks, goal):

    frontier = PriorityQueue()
    frontier.put((1.0,((), tuple(blocks))))

    solution = None

    n = 0
    while frontier:
        n += 1
        state = frontier.get()

        current_bag, available_blocks = state[1]

        # create a flatten list (transformed then into set) from current state in order to check a possible solution
        current_sol_set = set([x for sublist in current_bag for x in sublist])

        if len(current_sol_set)> 0 and len(current_sol_set)==len(goal):
            solution = current_bag
            #logging.info(f"Found a solution in {n:,} steps: {current_bag}")
            break

        for i, object in enumerate(available_blocks):

            new_state = (
                tuple((*current_bag, object)), # insert a new element on the current state
                tuple(available_blocks[:i] + available_blocks[i + 1 :]),
            )
            
            # taken into consideration a new state evaluate its cost
            current_bag_set = set([x for sublist in current_bag for x in sublist])
            new_cost = compute_cost(current_bag_set, object)

            if new_cost != -1:
                frontier.put((new_cost,new_state))

    return solution

In [7]:
import logging

logging.getLogger().setLevel(logging.INFO)

N = 20
blocks = problem(N,seed=42)
goal = [x for x in range(N)]
print(blocks)
sol = tree_search(blocks, goal)
print("solution", sol)

[[8, 4, 7], [0, 1, 2, 3, 6, 13, 17, 18], [0, 6, 16, 17, 19], [0, 5, 7, 8, 13, 14, 17, 18], [2, 3, 4, 6, 8, 10], [1, 3, 8, 11, 14, 19], [2, 3, 9, 11, 12, 17, 18, 19], [1, 2, 9, 7], [3, 5, 6, 7, 8, 11, 12, 14], [2, 5, 7, 8, 12, 14, 17, 19], [17, 10, 1, 7], [2, 6, 8, 10, 12, 15, 18], [4, 7, 8, 14, 17, 18], [4, 7, 11, 12, 15, 16, 18], [1, 3, 4, 5], [2, 8, 12, 13, 14, 16, 17, 19], [0, 3, 5, 8, 9, 10, 13, 14, 17], [8, 16, 5], [16, 9, 19, 6], [0, 5, 11, 16, 17], [0, 1, 3, 7, 9, 10, 11, 15], [18, 2, 15], [4, 5, 8, 13, 15, 16, 17, 19], [6, 9, 11, 12, 17], [2, 3, 7, 10, 14, 16], [17, 18, 7], [0, 1, 2, 7], [16, 10, 2, 7], [4, 6, 15, 17, 18], [3, 6, 7, 13, 15], [1, 3, 13, 14], [3, 6, 7, 10, 14, 17], [5, 7, 8, 13, 14], [0, 1, 2, 3, 5, 7, 14, 17]]
solution ([0, 3, 5, 8, 9, 10, 13, 14, 17], [4, 7, 11, 12, 15, 16, 18], [0, 1, 2, 3, 6, 13, 17, 18], [16, 9, 19, 6])


In [3]:
import logging

logging.getLogger().setLevel(logging.INFO)

for N in [5, 10 , 20, 100, 500, 1000]:
    blocks = problem(N,seed=42)
    goal = [x for x in range(N)]
    sol = tree_search(blocks, goal)
    logging.info(f"Found solution for N={N}: w={sum(len(_) for _ in sol)} (bloat={(sum(len(_) for _ in sol)-N)/N*100:.0f}%)")

INFO:root:Found solution for N=5: w=5 (bloat=0%)
INFO:root:Found solution for N=10: w=10 (bloat=0%)
INFO:root:Found solution for N=20: w=28 (bloat=40%)
INFO:root:Found solution for N=100: w=173 (bloat=73%)
INFO:root:Found solution for N=500: w=1320 (bloat=164%)
INFO:root:Found solution for N=1000: w=2893 (bloat=189%)
